In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from sentimentanalyser.utils.data import (
    Path, pad_collate, grandparent_splitter, random_splitter)
from sentimentanalyser.utils.data import parent_labeler
from sentimentanalyser.data.text import TextList, SplitData
from sentimentanalyser.utils.files import pickle_dump, pickle_load

/home/anukoolpurohit/anaconda3/envs/Torch10cuda/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
from sentimentanalyser.preprocessing.processor import TokenizerProcessor
from sentimentanalyser.preprocessing.processor import NuemericalizeProcessor
from sentimentanalyser.preprocessing.processor import CategoryProcessor
from sentimentanalyser.utils.data import read_wiki

In [4]:
from sentimentanalyser.utils.callbacks import sched_cos, combine_scheds
from sentimentanalyser.callbacks.training import (
    LRFind, CudaCallback, GradientClipping, RNNCustomTrainer)
from sentimentanalyser.callbacks.progress import ProgressCallback
from sentimentanalyser.callbacks.scheduler import ParamSchedulerCustom
from sentimentanalyser.callbacks.stats import AvgStatsCallback
from sentimentanalyser.callbacks.recorder import RecorderCustom
from sentimentanalyser.training.trainer import Trainer

In [5]:
from sentimentanalyser.optimizers import adam_opt
from sentimentanalyser.utils.metrics import accuracy
from sentimentanalyser.utils.callbacks import combine_scheds, sched_cos, cos_1cycle_anneal
from sentimentanalyser.utils.callbacks import create_phases

In [6]:
from sentimentanalyser.preprocessing.tokens import TOKENS

In [7]:
import torch
import numpy as np
import torch.nn.functional as F
from torch import nn
from functools import partial

In [8]:
path_imdb = Path("/home/anukoolpurohit/Documents/AnukoolPurohit/Datasets/imdb")
path_wiki = Path("/home/anukoolpurohit/Documents/AnukoolPurohit/Datasets/wikitext-103")
path_cache = Path('/home/anukoolpurohit/Documents/AnukoolPurohit/Models/WordEmbeddings')

In [9]:
path_model = Path('/home/anukoolpurohit/Documents/AnukoolPurohit/Models')

In [10]:
# """Uncomment to download the pretrained awd_lstm model from fastai"""
# path = path_model
# !wget http://files.fast.ai/models/wt103_tiny.tgz -P {path} --no-check-certificate
# !tar xf {path}/wt103_tiny.tgz -C {path}

In [11]:
proc_tok = TokenizerProcessor()
proc_num = NuemericalizeProcessor()

In [12]:
# tl_imdb_lm = TextList.from_files(path=path_imdb, folders=['train','test', 'unsup'])
# sd_imdb_lm = tl_imdb_lm.split_by_func(partial(random_splitter, p=[0.9, 0.1]))
# ll_imdb_lm = sd_imdb_lm.label_by_func(lambda x:0, proc_x=[proc_tok, proc_num])
# pickle_dump(ll_imdb_lm, 'dumps/variable/ll_imdb_lm.pickle')

In [13]:
ll_imdb_lm = pickle_load('dumps/variable/ll_imdb_lm.pickle')

In [14]:
bs, bptt = 64, 70
wiki_data = ll_imdb_lm.lm_databunchify(bs, bptt)

In [15]:
vocab = ll_imdb_lm.train.proc_x[1].vocab

In [16]:
tok_pad = vocab.index(TOKENS.PAD)

In [17]:
from sentimentanalyser.models.rnn import EncDecLanguageModel

In [18]:
def cross_entropy_flat(input, target):
    bs, sl = target.size()
    return F.cross_entropy(input.view(bs * sl, -1), target.view(bs * sl))

In [19]:
def accuracy_flat(input, target):
    bs, sl = target.size()
    return accuracy(input.view(bs * sl, -1), target.view(bs * sl))

In [20]:
def get_basic(Model, vocab):
    model = Model(len(vocab))
    loss_func = cross_entropy_flat
    opt = adam_opt()(model.parameters())
    return model, loss_func, opt

In [21]:
sched = combine_scheds([0.3, 0.7], [sched_cos(1e-4, 1e-3), sched_cos(1e-3, 3e-5)])

In [22]:
cbs = [partial(AvgStatsCallback, [accuracy_flat]),
       partial(ParamSchedulerCustom,'lr', [sched]),
       partial(GradientClipping, clip=0.1),
       ProgressCallback,
       CudaCallback,
       partial(RNNCustomTrainer, α=2., β=1.),
       RecorderCustom]

In [23]:
trainer = Trainer(wiki_data, *get_basic(EncDecLanguageModel, vocab), cb_funcs=cbs)

In [24]:
trainer.fit()

epoch,train_loss,train_accuracy_flat,valid_loss,valid_accuracy_flat,time
0,5.668839,0.169747,4.740767,0.240788,15:50
1,4.626917,0.246734,4.388263,0.270565,14:38
2,4.446017,0.262139,4.326300,0.276524,14:32
